In [1]:
!pip install -q langchain pypdf sentence-transformers chromadb langchain-openai python-dotenv



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

import os
load_dotenv()




In [7]:
# 문서 로드 및 처리
loaders = [
    WebBaseLoader("https://terms.naver.com/entry.naver?docId=926577&cid=51007&categoryId=51007"),
    WebBaseLoader("https://terms.naver.com/entry.naver?docId=926584&cid=51007&categoryId=51007"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load_and_split())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(docs)

In [11]:
# 임베딩 설정
model_name = "bespin-global/klue-sroberta-base-continue-learning-by-mnr"
ko_embedding = HuggingFaceEmbeddings(model_name=model_name)

# BM25 및 Chroma 리트리버 설정
bm25_retriever = BM25Retriever.from_documents(texts)
chroma_db = Chroma.from_documents(documents=texts, embedding=ko_embedding)
chroma_retriever = chroma_db.as_retriever(search_kwargs={"k": 2})

# 앙상블 리트리버 설정
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], 
    weights=[0.5, 0.5]
)
# OpenAI LLM 설정

openai_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature = 0,    openai_api_key=os.getenv("OPENAI_API_KEY")
)

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
# Conversational Retrieval Chain 생성
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=openai_llm,
    retriever=ensemble_retriever,
    return_source_documents=True
)

# 대화 상태 초기화
conversation_history = []

# 챗봇 루프
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break
    
    # 대화 히스토리에 새로운 질문 추가
    conversation_history.append(("user", query))
    
    # qa_chain에 chat_history를 전달할 때 올바른 형식으로 전달
    result = qa_chain({"question": query, "chat_history": conversation_history})
    response = result['answer']
    
    print(f"Bot: {response}")
    
    # 대화 히스토리 업데이트
    conversation_history.append(("assistant", response))

Bot: 감기의 증상으로는 재채기, 코막힘, 콧물, 인후통, 기침, 미열, 두통 및 근육통 등이 있습니다. 증상은 감기 바이러스에 노출된 후 1~3일 후에 나타나며, 성인에게는 열이 드물거나 미열에 그치는 경우가 많고, 소아에게서는 발열 증상이 흔하게 나타납니다.
Bot: 인플루엔자는 인플루엔자 바이러스에 의한 급성 호흡기 질환입니다. 이 질환은 상부 호흡기계(코, 목)나 하부 호흡기계(폐)를 침범하며, 갑작스러운 고열, 두통, 근육통, 전신 쇠약감과 같은 전반적인 신체 증상을 동반합니다. 인플루엔자는 전염성이 강하고, 노인, 소아, 및 기저질환이 있는 사람이 감염될 경우 사망률이 증가하고 합병증이 발생할 수 있습니다. 매년 겨울에 소규모로 유행하며, 새로운 종류의 인플루엔자 바이러스가 나타나면 대유행을 일으킬 수 있습니다.
Bot: 감기의 진료과는 일반적으로 내과나 가정의학과에서 진료를 받습니다. 감기는 주로 상부 호흡기계에 영향을 미치는 질환이기 때문에 이러한 과에서 진료를 받는 것이 적합합니다.
Bot: 감기와 관련된 질병은 다음과 같습니다:

- 상기도감염
- 비염
- 부비동염
- 인후두염
- 기관지염
- 폐렴
- 천식
- 독감
Bot: 감기와 관련된 질병으로는 독감이 있습니다. 독감은 감기와 일부 증상이 비슷할 수 있지만, 원인 바이러스가 다르며 증상이나 합병증, 치료법도 다릅니다. 또한, 감기는 상부 호흡기계의 감염으로, 재채기, 코막힘, 콧물, 인후통, 기침 등의 증상이 나타납니다.
Bot: 제가 이전에 질문한 내용을 요약할 수는 없습니다. 하지만 독감과 감기에 대한 진단, 치료, 경과 및 합병증, 예방 방법에 대한 정보를 제공할 수 있습니다. 필요하신 정보가 있다면 말씀해 주세요.
Bot: 저는 요약할 수 있습니다. 어떤 내용을 요약해 드릴까요?
